In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
import re
import collections
import random
import math

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [81]:
pd.set_option('display.max_column', None)
train_input = pd.read_csv('train.csv')
test_input = pd.read_csv('test.csv')
print("train_input shape is: ", np.shape(train_input))
print("test_input shape is: ", np.shape(test_input))

train_input shape is:  (4743, 2)
test_input shape is:  (1701, 2)


In [82]:
def label_tweet(input_set):
    handle = input_set['handle']
    # put it into an array named label, where 0 represents HillaryClinton, 1 represents readDonaldTrump
    label = []
    for i in range(len(handle)):
        if handle[i] == "HillaryClinton":
            label.append(0)
        if handle[i] == "realDonaldTrump":
            label.append(1)
    label = np.asarray(label)
    return label

train_label = label_tweet(train_input)
print(train_label[:10])

[0 0 0 0 1 0 0 0 1 1]


In [83]:
train_corpus = train_input['tweet'].as_matrix()
print(np.shape(train_corpus))
print(train_corpus[:5])
test_corpus = test_input['tweet'].as_matrix()
print(np.shape(test_corpus))

(4743,)
[ 'The question in this election: Who can put the plans into action that will make your life better? https://t.co/XreEY9OicG'
 'Last night, Donald Trump said not paying taxes was "smart." You know what I call it? Unpatriotic. https://t.co/t0xmBfj7zF'
 "If we stand together, there's nothing we can't do. \n\nMake sure you're ready to vote: https://t.co/tTgeqxNqYm https://t.co/Q3Ymbb7UNy"
 "Both candidates were asked about how they'd confront racial injustice. Only one had a real answer. https://t.co/sjnEokckis"
 'Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc']
(1701,)


In [84]:
# Load the stopwords
stop_words = stopwords.words('english')
# 'https' seems useless, so I add it to stop_words
stop_words.append(u'https')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [85]:
# Tokenize
def tokenization(text):
    tokens=[]
    for word in nltk.word_tokenize(text):
        # skip all the websites, punctuations, pure digits
        if not re.match('[//]', word) and re.search('[a-zA-Z]', word) and word.lower() not in stop_words:
            tokens.append(word.lower())
    return tokens

# Tokenize training set
train_corpus_tokenized = []
for i in train_corpus:
    train_corpus_tokenized.append(' '.join(tokenization(i)))
    
print(len(train_corpus_tokenized))
print(train_corpus_tokenized[:5])

# Tokenize testing set
test_corpus_tokenized = []
for i in test_corpus:
    test_corpus_tokenized.append(' '.join(tokenization(i)))

print(len(test_corpus_tokenized))
print(test_corpus_tokenized[:5])

4743
['question election put plans action make life better', 'last night donald trump said paying taxes smart know call unpatriotic', "stand together 's nothing ca n't make sure 're ready vote", "candidates asked 'd confront racial injustice one real answer", 'join 3pm rally tomorrow mid-america center council bluffs iowa tickets']
1701
["could n't proud hillaryclinton vision command last night 's debate showed 's ready next potus", "election important sit go make sure 're registered nationalvoterregistrationday -h", 'government people join movement today', "national voterregistrationday make sure 're registered vote makeamericagreatagain…", 'great afternoon little havana hispanic community leaders thank support imwithyou']


In [86]:
train_tokenized_word = []
for i in range(len(train_corpus_tokenized)):
    train_tokenized_word.append(tf.compat.as_str(train_corpus_tokenized[i]).split())
print(len(train_tokenized_word))
print(train_tokenized_word[:5])

vocabulary_size = 10000

test_tokenized_word = []
for i in range(len(test_corpus_tokenized)):
    test_tokenized_word.append(tf.compat.as_str(test_corpus_tokenized[i]).split())
print(len(test_tokenized_word))
print(test_tokenized_word[:5])

4743
[['question', 'election', 'put', 'plans', 'action', 'make', 'life', 'better'], ['last', 'night', 'donald', 'trump', 'said', 'paying', 'taxes', 'smart', 'know', 'call', 'unpatriotic'], ['stand', 'together', "'s", 'nothing', 'ca', "n't", 'make', 'sure', "'re", 'ready', 'vote'], ['candidates', 'asked', "'d", 'confront', 'racial', 'injustice', 'one', 'real', 'answer'], ['join', '3pm', 'rally', 'tomorrow', 'mid-america', 'center', 'council', 'bluffs', 'iowa', 'tickets']]
1701
[['could', "n't", 'proud', 'hillaryclinton', 'vision', 'command', 'last', 'night', "'s", 'debate', 'showed', "'s", 'ready', 'next', 'potus'], ['election', 'important', 'sit', 'go', 'make', 'sure', "'re", 'registered', 'nationalvoterregistrationday', '-h'], ['government', 'people', 'join', 'movement', 'today'], ['national', 'voterregistrationday', 'make', 'sure', "'re", 'registered', 'vote', 'makeamericagreatagain…'], ['great', 'afternoon', 'little', 'havana', 'hispanic', 'community', 'leaders', 'thank', 'support',

In [87]:
cnt = collections.Counter()
for i in range(len(train_tokenized_word)):
    for word in train_tokenized_word[i]:
        cnt[word] += 1

print(len(cnt))

8507


In [88]:
def build_dataset(cnt, words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(cnt.most_common(n_words - 1))
#     print count[:20]
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = []
    data_all = []
    unk_count = 0
    for i in range(len(words)):
        inner_data = []
        for word in words[i]:
            index = dictionary.get(word, 0)
            if index == 0:  # dictionary['UNK']
                unk_count += 1
            inner_data.append(index)
            data_all.append(index)
        data.append(inner_data)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, data_all, count, dictionary, reversed_dictionary

train_x, data_all, count, dictionary, reverse_dictionary = build_dataset(cnt, train_tokenized_word, vocabulary_size)

In [107]:
test = []
for sentence in test_tokenized_word:
    cur = []
    for word in sentence:
        if(word in dictionary):
            cur.append(dictionary[word])
        else:
            cur.append(0)
    test.append([cur,[-1, -1]])

print(len(test))
print(test[:5])
test_length = len(test)
print(test_length)

1701
[[[92, 6, 120, 56, 1150, 0, 44, 83, 2, 99, 1387, 2, 199, 162, 32], [-1, -1]], [[104, 271, 1683, 51, 12, 145, 75, 600, 1661, 82], [-1, -1]], [[1341, 9, 43, 207, 26], [-1, -1]], [[116, 0, 12, 145, 75, 600, 22, 0], [-1, -1]], [[5, 914, 241, 0, 2898, 622, 497, 4, 68, 169], [-1, -1]]]
1701


In [90]:
train_all = [[train_x[i], [train_label[i], 1-train_label[i]]] for i in range(0, len(train_x))]
# train_all = [[train_x[i], [train_label[i]]] for i in range(0, len(train_x))]

r_index = list(range(len(train_all)))
random.shuffle(r_index)
train = [train_all[i] for i in r_index[:int(len(r_index)*0.8)]]
valid = [train_all[i] for i in r_index[int(len(r_index)*0.8):]]

print(len(train))
print(train[:5])
print(len(valid))
print(valid[:5])
print(len(count))

3794
[[[24, 2, 12, 708, 22, 909, 24, 121, 39, 618, 196, 172, 355, 2965, 16], [0, 1]], [[4, 7203, 5, 77, 7204, 50, 39, 155, 14], [1, 0]], [[7, 1, 5201, 10, 507, 1742, 463, 198], [0, 1]], [[301, 473, 178, 92, 6, 4686], [0, 1]], [[5, 569, 1017, 219, 183, 28, 3, 6966, 332, 133, 381], [1, 0]]]
949
[[[151, 48, 140, 315, 759, 12, 5, 624, 11, 32], [0, 1]], [[3, 15, 2413, 5629, 237, 46, 533, 5630, 150, 5631, 3254], [0, 1]], [[280, 244, 442, 247, 764, 5605, 41, 1072, 5606, 5607, 1132, 371], [0, 1]], [[1086, 35, 1356, 141, 1999, 27, 402, 101, 310, 320, 3491, 1015], [1, 0]], [[132, 625, 5040, 3092, 1066, 5041, 3, 474, 356, 5042, 32], [0, 1]]]
8508


In [91]:
class SimpleDataIterator():
    def __init__(self, df):
        self.df = df
        self.size = len(self.df)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.df)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            print("SimpleDataIterator epoch : ", self.epochs)
            self.shuffle()
        res = self.df[self.cursor:self.cursor+n]
        self.cursor += n
        return res

In [92]:
data = SimpleDataIterator(valid)
d = data.next_batch(500)
print('Input sequences\n', d[:5], end='\n\n')

Input sequences
 [[[581, 3407, 122, 62, 6, 824, 880, 329, 209, 677, 646, 348, 223, 547], [0, 1]], [[449, 215, 613, 50, 104, 6, 1522, 58, 4720, 994, 2084], [0, 1]], [[7229, 2173], [0, 1]], [[20, 9, 182, 22, 249, 98, 3355, 7829, 1050, 192, 1384], [1, 0]], [[59, 159, 64, 59, 3, 48, 32], [0, 1]]]



In [93]:
class PaddedDataIterator(SimpleDataIterator):
    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df[self.cursor:self.cursor+n]
        self.cursor += n

        # Pad sequences with 0s so they are all the same length
        max_len = 0
        for row in res:
            if len(row[0]) > max_len:
                max_len = len(row[0])
        seqlen = np.array([max_len for i in range(len(res))])
        ret = []
        label = []
        for row in res:
            ret += [row[0] + [0]*(max_len-len(row[0]))]
            label.append(row[1][0])
        x = np.array(ret)
        y = np.array(label)

        return x, y, seqlen

In [94]:
data = PaddedDataIterator(train)
d = data.next_batch(3)
print('Input sequences\n', d[0], end='\n\n')
print(d[1])
print(d[1].shape)
print(d[2])

Input sequences
 [[  36 3580  832  536 1138  914    6   24 1015   74   22    0]
 [5351    8 5352 5353   31  649   46  262    9  579 1128  372]
 [ 211  118   52   58 3475 1039    0    0    0    0    0    0]]

[1 0 0]
(3,)
[12 12 12]


In [108]:
batch_size = 256

def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def build_graph(
    vocab_size = len(dictionary),
    state_size = 64,
    batch_size = 256,
    num_classes = 6):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
#     keep_prob = tf.constant(1.0)
#     print("====",x.shape)
#     print("====",y.shape)
#     print("====",seqlen.shape)

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    cell = tf.nn.rnn_cell.GRUCell(state_size)
    init_state = tf.get_variable('init_state', [1, state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                 initial_state=init_state)

    # Add dropout, as the model otherwise quickly overfits
#     rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)

    """
    Obtain the last relevant output. The best approach in the future will be to use:

        last_rnn_output = tf.gather_nd(rnn_outputs, tf.pack([tf.range(batch_size), seqlen-1], axis=1))

    which is the Tensorflow equivalent of numpy's rnn_outputs[range(30), seqlen-1, :], but the
    gradient for this op has not been implemented as of this writing.

    The below solution works, but throws a UserWarning re: the gradient.
    """
    idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)

    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
#     print(correct.shape)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
#     print(accuracy.shape)

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
#         'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }

In [101]:
def train_graph(graph, batch_size = 256, num_epochs = 40, iterator = PaddedDataIterator):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        tr = iterator(train)
        tv = iterator(valid)
#         te = iterator(test)

        step, accuracy = 0, 0
        tr_losses, tv_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
#             print("----",batch[0].shape)
#             print("----",batch[1].shape)
#             print("----",batch[2].shape)
#             dropout_parameter=np.array(0.6)
#             print("----",dropout_parameter.shape)
#             feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: dropout_parameter}
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                tv_epoch = tv.epochs
                while tv.epochs == tv_epoch:
                    step += 1
                    batch = tv.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                tv_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- tv:", tv_losses[-1])
                
            

    return tr_losses, te_losses

In [102]:
g = build_graph()
tr_losses, te_losses = train_graph(g)

/home/ziyi/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Accuracy after epoch 1  - tr: 0.500260416667 - tv: 0.5068359375
Accuracy after epoch 2  - tr: 0.534040178571 - tv: 0.513020833333
Accuracy after epoch 3  - tr: 0.498604910714 - tv: 0.509114583333
Accuracy after epoch 4  - tr: 0.499441964286 - tv: 0.50390625
Accuracy after epoch 5  - tr: 0.497767857143 - tv: 0.46875
Accuracy after epoch 6  - tr: 0.5 - tv: 0.506510416667
Accuracy after epoch 7  - tr: 0.492745535714 - tv: 0.51953125
Accuracy after epoch 8  - tr: 0.494419642857 - tv: 0.514322916667
Accuracy after epoch 9  - tr: 0.497767857143 - tv: 0.5
Accuracy after epoch 10  - tr: 0.501953125 - tv: 0.489583333333
Accuracy after epoch 11  - tr: 0.499441964286 - tv: 0.54296875
Accuracy after epoch 12  - tr: 0.502232142857 - tv: 0.490885416667
Accuracy after epoch 13  - tr: 0.498604910714 - tv: 0.524739583333
Accuracy after epoch 14  - tr: 0.571149553571 - tv: 0.688802083333
Accuracy after epoch 15  - tr: 0.625279017857 - tv: 0.513020833333
Accuracy after epoch 16  - tr: 0.754185267857 - tv

In [103]:
test[:10]

[[[48, 160, 153, 2457, 1283, 650, 614, 115, 557], [-1, -1]],
 [[4, 6734, 6735], [-1, -1]],
 [[93, 37, 330, 3, 2, 378, 3357, 9, 361, 25, 0, 475, 639, 879, 834], [-1, -1]],
 [[2180, 298, 5990, 617, 0, 368, 51, 918, 135, 530, 397], [-1, -1]],
 [[62, 6, 42, 535, 1637, 1175, 548, 34, 0, 682, 9, 479, 212, 9, 36, 13],
  [-1, -1]],
 [[4, 263, 14], [-1, -1]],
 [[7671, 33, 9, 152, 300, 65, 65, 65, 28, 3, 1611, 23, 25, 1056, 804],
  [-1, -1]],
 [[0, 1, 2, 0, 0, 2374, 1529, 473, 266, 229, 6431], [-1, -1]],
 [[913, 3308, 65, 3007, 558, 10, 280, 108, 2, 5, 6304, 6139, 13, 64, 0],
  [-1, -1]],
 [[587, 0, 444, 1115, 67, 2291, 582, 17, 1904, 383, 6727, 9, 298, 1404, 1978],
  [-1, -1]]]

In [115]:
def align(data):
    max_len = 0
    for row in data:
        if len(row) > max_len:
            max_len = len(row)
    ret = []
    for row in data:
        ret += [row + [0]*(max_len-len(row))]
    return ret, max_len

In [114]:
# test_length = 1701
print(test_length)
print(batch_size)
print(test_length/batch_size)

train_list = []
for i in range(math.ceil(test_length/batch_size)):
    print(i)
    

# test_align, max_len = align(test)
# print(len(test_align))

1701
256
6.64453125
0
1
2
3
4
5
6
